In [1]:
!pip install datasets
!pip install peft
!pip install transformers
!pip install gradio
!pip uninstall bitsandbytes -y
!pip install --upgrade bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import Dataset
import json
from peft import LoraConfig, get_peft_model

def load_dataset():
    data = [
    {"incorrect": "Fix this incorrect equation: 8 ÷ 2(2+2) = 1", "correct": "The correct answer is 16. Solution: Parentheses first (2+2) = 4, then left-to-right division & multiplication: 8 ÷ 2 × 4 = 4 × 4 = 16."},
    {"incorrect": "Fix this incorrect equation: 5² = 10", "correct": "The correct answer is 25. Exponents before multiplication: 5² = 5 × 5 = 25."},
    {"incorrect": "Fix this incorrect equation: 10 - 3 × 2 = 14", "correct":  "The correct answer is 4. Multiplication before subtraction: 3 × 2 = 6, then 10 - 6 = 4."},
    {"incorrect": "Fix this incorrect equation: (5 + 3)² = 40", "correct": "The correct answer is 64. Solution: (5 + 3)² = 64. Parentheses first, then exponentiation!"},
    {"incorrect": "Fix this incorrect equation: 6 ÷ 2(1+2) = 1", "correct": "6 ÷ 2(1+2) = 9. Follow the correct order of operations!"},
    {"incorrect": "Fix this incorrect equation: √16 + 4 × 2 = 12", "correct": "√16 + 4 × 2 = 12. Square root first, then multiplication!"},
    {"incorrect": "Fix this incorrect equation: 3 + 4 × 2 = 14", "correct": "3 + 4 × 2 = 11. Multiplication before addition!"},
    {"incorrect": "Fix this incorrect equation: 100 ÷ 5 × 2 = 10", "correct": "100 ÷ 5 × 2 = 40. Perform left-to-right operations correctly!"},
    {"incorrect": "Fix this incorrect equation: 7 + 3 × (8 - 5) = 30", "correct": "7 + 3 × (8 - 5) = 16. Parentheses first, then multiplication!"},
    {"incorrect": "Fix this incorrect equation: 2 × 3² = 6", "correct":  "The correct answer is 18. Solution: 2 × 3² = 18. Exponents before multiplication!"},
    {"incorrect": "Fix this incorrect equation: 9 - 2 × 3 = 21", "correct": "9 - 2 × 3 = 3. Multiplication before subtraction!"},
    {"incorrect": "Fix this incorrect equation: (4 + 2) × 3 = 10", "correct": "(4 + 2) × 3 = 18. Parentheses first!"},
    {"incorrect": "Fix this incorrect equation: √25 + 5 × 2 = 10", "correct": "√25 + 5 × 2 = 15. Square root first, then multiplication!"},
    {"incorrect": "Fix this incorrect equation: 12 ÷ 4 + 2 = 1", "correct": "12 ÷ 4 + 2 = 5. Division first, then addition!"},
    {"incorrect": "Fix this incorrect equation: 8 + 4 ÷ 2 = 6", "correct": "8 + 4 ÷ 2 = 10. Division before addition!"},
    {"incorrect": "Fix this incorrect equation: (10 - 4) ÷ 2 = 1", "correct": "(10 - 4) ÷ 2 = 3. Parentheses first, then division!"},
    {"incorrect": "Fix this incorrect equation: 2 + 2 × 2 = 6", "correct": "2 + 2 × 2 = 6. Correct! Multiplication before addition!"},
    {"incorrect": "Fix this incorrect equation: 1 + 1 × 0 + 1 = 0", "correct": "1 + 1 × 0 + 1 = 2. Multiplication before addition!"},
    {"incorrect": "Fix this incorrect equation: 15 ÷ 3 × 2 = 2.5", "correct": "15 ÷ 3 × 2 = 10. Left-to-right operations!"},
    {"incorrect": "Fix this incorrect equation: 20 - 5 × 3 = 45", "correct": "20 - 5 × 3 = 5. Multiplication before subtraction!"},
    {"incorrect": "Fix this incorrect equation: (6 + 2)² ÷ 4 = 8", "correct": "(6 + 2)² ÷ 4 = 16. Parentheses, exponents, then division!"},
]

    with open("math_meme_dataset.json", "w") as f:
        json.dump(data, f)
    return Dataset.from_list(data)

def load_model(finetuned=False):
    model_name = "./math_meme_finetuned" if finetuned else "Qwen/Qwen2-1.5B"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token = tokenizer.eos_token

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.bfloat16,  # Use bfloat16 to reduce memory
        device_map="auto").to(device)

    model.gradient_checkpointing_enable()

    return model, tokenizer

def apply_lora(model):
    lora_config = LoraConfig(
        r=8,
        lora_alpha=32,
        lora_dropout=0.1,
        bias="none",
        target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],  # Add more layers
        modules_to_save=["lm_head"]
    )
    model = get_peft_model(model, lora_config)
    model.print_trainable_parameters()
    return model



def formatting_prompts_func(examples,tokenizer):
    texts = [prob + " -> " + sol + tokenizer.eos_token for prob, sol in zip(examples["incorrect"], examples["correct"])]
    return {"text": texts}

# dataset = dataset.map(formatting_prompts_func, batched=True)

def tokenize_function(examples,tokenizer):
    model_inputs = tokenizer(examples["incorrect"], padding="max_length", truncation=True, max_length=256)
    model_inputs["labels"] = model_inputs["input_ids"].copy()
    return model_inputs


def train_model():
    dataset = load_dataset()
    model, tokenizer = load_model()
    model = apply_lora(model)

    dataset = dataset.map(lambda x: formatting_prompts_func(x, tokenizer), batched=True)
    tokenized_data = dataset.map(lambda x: tokenize_function(x, tokenizer), batched=True)

    training_args = TrainingArguments(
        output_dir="./math_meme_finetuned",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        num_train_epochs=3,
        learning_rate=2e-4,
        logging_dir="./logs",
        report_to="none",
        save_strategy="epoch"
    )

    trainer = Trainer(
        model=model,
        args=training_args,

        train_dataset=tokenized_data,
        tokenizer=tokenizer
    )
    trainer.train()
    model.save_pretrained("./math_meme_finetuned")
    tokenizer.save_pretrained("./math_meme_finetuned")


In [13]:

def test_model(test_cases):
    model, tokenizer = load_model(finetuned=True)
    model.eval()
    test_cases= test_cases
    device = "cuda" if torch.cuda.is_available() else "cpu"

    input_text= test_cases + '->'
    inputs = tokenizer(input_text, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=256)
    corrected = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected


torch.cuda.empty_cache()  # Clears cached memory
train_model()


In [25]:
import gradio as gr
# Gradio Interface
def create_gradio_interface():
    with gr.Blocks(
        title="Math Equation Corrector",
        theme=gr.themes.Soft(),  # A sleek, modern theme
        css="""
            .gradio-container {background: linear-gradient(135deg, #f5f7fa, #c3cfe2); font-family: 'Arial', 'sans-serif';}
            h1 {color: #2c3e50; text-align: center; font-size: 3rem; margin-bottom: 20px;}
            title{color: #2d3e50; text-align: center; font-size: 7rem; margin-bottom: 20px;}
            .output-text {font-size: 1.2em; padding: 15px; border-radius: 10px; background: #ecf0f1; box-shadow: 0 4px 8px rgba(0,0,0,0.1);}
        """
    ) as demo:
        gr.Markdown("🧠 Math Equation Corrector")
        gr.Markdown("Enter an incorrect equation, and watch the magic happen! ✨")

        with gr.Row():
            with gr.Column(scale=1):
                input_box = gr.Textbox(
                    label="Enter Incorrect Equation",
                    placeholder="e.g., 6 ÷ 3 = 1",
                    lines=2,
                    elem_classes="input-text"
                )
                submit_btn = gr.Button("Correct It!", variant="primary")

            with gr.Column(scale=1):
                output_box = gr.Markdown(
                    label="Corrected Result",
                    elem_classes="output-text"
                )

        # Add examples for users to try
        examples = gr.Examples(
            examples=[
                "Fix this incorrect equation: 6 ÷ 3 = 1 ",
                "Fix this incorrect equation: 8 × 2 = 14 ",
                "Fix this incorrect equation: (8 + 4)÷2 = 40 ",
                "Fix this incorrect equation: (8 × 12)+3-4 = 14 ",

            ],
            inputs=input_box
        )

        # Connect the button to the prediction function
        submit_btn.click(
            fn=test_model,
            inputs=input_box,
            outputs=output_box
        )

    return demo

# Launch the app
if __name__ == "__main__":
    # torch.cuda.empty_cache()  # Uncomment if you need to clear GPU memory
    interface = create_gradio_interface()
    interface.launch(
        server_name="0.0.0.0",  # Accessible on local network
        server_port=6666,
        share=True,  # Generates a public URL for sharing
        debug=True
    )

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://dd4a9e0aab7713d11e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.
Both `max_new_tokens` (=2048) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:6666 <> https://dd4a9e0aab7713d11e.gradio.live
